### 1. Load data

Data on social values were obtained (and then transformed) from [World Values Survey](https://www.worldvaluessurvey.org/wvs.jsp) under this considerations:
- Latest survey conducted from 2016 to 2022 with code S025. If the country has not conducted any surveys during this period, it is not included.
- Some country names have been changed to make them recognizable by Tableau.
- Some continental regions are debatable in their membership and have been categorized by me. In this regard, the following factors have been taken into account: geographical location, cultural similarity or good relations and a balanced distribution by groups (e.g., North America & Oceania).

In [4]:
import pandas as pd

df = pd.read_csv('world_values.csv')
df['population 2022'] = df['population 2022'].astype(int)
df.head()

,Country,tradition_secular_axis,survival_selfexpression_axis,Year,population 2022,Region,Population Group,size
0,Haiti,-0.47,-0.14,2016,11584996,Central & South America,<25M,5
1,Greece,0.20,-0.27,2017,10426919,Europe,<25M,5
2,Finland,0.52,0.75,2017,5556106,Europe,<25M,5
3,Denmark,0.65,0.92,2017,5903037,Europe,<25M,5
4,Switzerland,0.44,0.71,2017,8775760,Europe,<25M,5


In [5]:
# Obviously, the sum of the population in 2022 is not equal to the world population
# Data from: https://data.worldbank.org/indicator/SP.POP.TOTL?end=2022
print(df['population 2022'].sum())

5245048709


In [ ]:
print(df.groupby('Region')['population 2022'].sum())

Region
Africa                      462083248
Central & South America     468136660
Central Asia                272911319
East Asia                  2418256079
East Europe & West Asia     198871660
Europe                      541338895
Middle East                 352604657
North America & Oceania     530846191
Name: population 2022, dtype: int64


### 2. Categorization and its values for the pie chart

In [ ]:
df2 = df.copy()
# Index based on the avg since both extremes represent the same category or set of values
df2['Conservative_Liberal'] = (df['tradition_secular_axis'] + df['survival_selfexpression_axis']) / 2

# Create vars to use them latter
df2_minusOne_to_minusZeroFive = df2[(df2['Conservative_Liberal'] >= -1) &
                                    (df2['Conservative_Liberal'] <= -0.5)]
df2_minusZeroFive_to_zero = df2[(df2['Conservative_Liberal'] >= -0.5) &
                                (df2['Conservative_Liberal'] <= 0)]
df2_zero_to_zeroFive = df2[(df2['Conservative_Liberal'] >= 0) &
                            (df2['Conservative_Liberal'] <= 0.5)]
df2_zeroFive_to_one = df2[(df2['Conservative_Liberal'] >= 0.5) &
                            (df2['Conservative_Liberal'] <= 1)]

total_population_by_region = df.groupby('Region')['population 2022'].sum().to_dict()

global_polulation = df['population 2022'].sum()

# Bc its not necesary to run a server, print to paste in .js 
print("-1 to -0.5: ", df2_minusOne_to_minusZeroFive['population 2022'].sum() / global_polulation * 100)
print("-1 to -0.5 total: ", df2_minusOne_to_minusZeroFive['population 2022'].sum())
print("\n-0.5 to 0: ", df2_minusZeroFive_to_zero['population 2022'].sum() / global_polulation * 100)
print("-0.5 to 0 total: ", df2_minusZeroFive_to_zero['population 2022'].sum())
print("\n0 to 0.5: ", df2_zero_to_zeroFive['population 2022'].sum() / global_polulation * 100)
print("0 to 0.5 total: ", df2_zero_to_zeroFive['population 2022'].sum())
print("\n0.5 to 1: ", df2_zeroFive_to_one['population 2022'].sum() / global_polulation * 100)
print("0.5 to 1 total: ", df2_zeroFive_to_one['population 2022'].sum())

-1 to -0.5:  25.585883477064197
-1 to -0.5 total:  1341992051

-0.5 to 0:  26.868658561393733
-0.5 to 0 total:  1409274229

0 to 0.5:  40.13080233913229
0 to 0.5 total:  2104880130

0.5 to 1:  7.647715612472876
0.5 to 1 total:  401126409


### 3. Bar plot with plotly to html

In [29]:
import pandas as pd
import plotly.express as px

def prepare_data(df, group_name, color):
    population_by_region = df.groupby("Region")["population 2022"].sum()
    percentage_population = (population_by_region / pd.Series(total_population_by_region)) * 100
    percentage_population_df = percentage_population.reset_index()
    percentage_population_df.columns = ["Region", "Percentage"]
    
    hover_text_by_region = df.groupby("Region").apply(
        lambda x: "<br>".join(
            f"{country}: {liberal}" 
            for country, liberal in zip(x['Country'], x['Conservative_Liberal'].round(2))
        ),
        include_groups=False
    ).reset_index(name="hover_text")
    
    percentage_population_df = percentage_population_df.merge(hover_text_by_region, on="Region")
    percentage_population_df["Group"] = group_name
    percentage_population_df["Color"] = color
    return percentage_population_df

data_conservative = prepare_data(df2_minusOne_to_minusZeroFive, "Conservative", "#3498db")
data_moderate_conservative = prepare_data(df2_minusZeroFive_to_zero, "Moderate Conservative", "#60a7d7")
data_moderate_liberal = prepare_data(df2_zero_to_zeroFive, "Moderate Liberal", "#de8378")
data_liberal = prepare_data(df2_zeroFive_to_one, "Liberal", "#e74c3c")

all_data = pd.concat([
    data_liberal,
    data_moderate_liberal,
    data_moderate_conservative,
    data_conservative
])

fig = px.bar(
    all_data,
    y="Region",
    x="Percentage",
    color="Group",
    color_discrete_map={
        "Liberal": "#e74c3c",
        "Moderate Liberal": "#de8378",
        "Moderate Conservative": "#60a7d7",
        "Conservative": "#3498db",
    },
    orientation='h',
    hover_data={"Percentage": ':.2f'},
    custom_data=["hover_text"]
)

fig.update_traces(
    hovertemplate='%{customdata[0]}<extra></extra>',
    marker_line_width=0,
    hoverlabel=dict(bgcolor='#141313', font_color='white', font_size=16, font_family='sans-serif', bordercolor='rgba(0, 0, 0, 0)'),
)

fig.update_layout(
    barmode='stack',
    showlegend=True,
    xaxis_title="Percentage of Region Population (%)",
    xaxis_range=[0, 100],
    yaxis_title=None,
    margin=dict(t=30, b=35, l=50, r=30),
    paper_bgcolor='rgba(20, 20, 20, 0.01)',
    plot_bgcolor='rgba(20, 20, 20, 0.01)',
    font=dict(family='sans-serif', color='white', size=16, weight='bold'),
    xaxis=dict(showgrid=False)
)

fig.show()
fig.write_html("liberal_conservative.html",config={"displayModeBar": False})